In [1]:
import blaze as bz

/Users/yhtgrace/anaconda3/lib/python3.5/site-packages/flask/exthook.py:71: ExtDeprecationWarning: Importing flask.ext.cors is deprecated, use flask_cors instead.
  .format(x=modname), ExtDeprecationWarning


Re-implement the sql query for patient ICU stays. Combines admissions, patients and icustays.

In [18]:
admissions = bz.data("../data/mimiciii/ADMISSIONS.csv")
patients = bz.data("../data/mimiciii/PATIENTS.csv")
icustays = bz.data("../data/mimiciii/ICUSTAYS.csv")

Patient-level factors: 

In [48]:
patients_ = bz.transform(patients,
    subject_id = patients.SUBJECT_ID, 
    gender = patients.GENDER, 
    dob = patients.DOB
)[[
    'subject_id', 
    'gender', 
    'dob'
]]
patients_.peek()

,subject_id,gender,dob
0,249,F,2075-03-13
1,250,F,2164-12-27
2,251,M,2090-03-15
3,252,M,2078-03-06
4,253,F,2089-11-26
5,255,M,2109-08-05
6,256,M,2086-07-31
7,257,F,2031-04-03
8,258,F,2124-09-19
9,260,F,2105-03-23


Hospital-level factors

In [74]:
admissions_ = admissions[admissions['HAS_CHARTEVENTS_DATA'] == 1]
admissions_ = bz.transform(admissions_, 
    admittime = admissions_.ADMITTIME, 
    dischtime = admissions_.DISCHTIME, 
    los_hospital = admissions_.DISCHTIME - admissions_.ADMITTIME,
    ethnicity = admissions_.ETHNICITY, 
    admission_type = admissions_.ADMISSION_TYPE, 
    hospital_expire_flag = admissions_.HOSPITAL_EXPIRE_FLAG,
    hadm_id = admissions_.HADM_ID,
)[[
    'admittime', 
    'dischtime',
    'los_hospital',
    'admission_type', 
    'hospital_expire_flag',
    'hadm_id'
]]
admissions_.peek()

,admittime,dischtime,los_hospital,admission_type,hospital_expire_flag,hadm_id
0,2196-04-09 12:26:00,2196-04-10 15:54:00,1 days 03:28:00,EMERGENCY,0,165315
1,2153-09-03 07:15:00,2153-09-08 19:10:00,5 days 11:55:00,ELECTIVE,0,152223
2,2157-10-18 19:34:00,2157-10-25 14:00:00,6 days 18:26:00,EMERGENCY,0,124321
3,2139-06-06 16:14:00,2139-06-09 12:48:00,2 days 20:34:00,EMERGENCY,0,161859
4,2160-11-02 02:06:00,2160-11-05 14:55:00,3 days 12:49:00,EMERGENCY,0,129635
5,2126-05-06 15:16:00,2126-05-13 15:00:00,6 days 23:44:00,EMERGENCY,0,197661
6,2191-11-30 22:16:00,2191-12-03 14:45:00,2 days 16:29:00,NEWBORN,0,134931
7,2177-09-01 07:15:00,2177-09-06 16:00:00,5 days 08:45:00,ELECTIVE,0,162569
8,2172-10-14 14:17:00,2172-10-19 14:37:00,5 days 00:20:00,URGENT,0,104557
9,2108-08-22 23:27:00,2108-08-30 15:00:00,7 days 15:33:00,EMERGENCY,1,128652


ICU-level factors: 

In [75]:
icustays_ = bz.transform(icustays, 
    intime = icustays.INTIME, 
    outtime = icustays.OUTTIME, 
    los_icu = icustays.OUTTIME - icustays.INTIME,
    hadm_id = icustays.HADM_ID,
    subject_id = icustays.SUBJECT_ID,
)[[
    'intime', 
    'outtime',
    'los_icu',
    'hadm_id',
    'subject_id',
]]
icustays_.peek()

,intime,outtime,los_icu,hadm_id,subject_id
0,2198-02-14 23:27:38,2198-02-18 05:26:11,3 days 05:58:33,110404,268
1,2170-11-05 11:05:29,2170-11-08 17:46:57,3 days 06:41:28,106296,269
2,2128-06-24 15:05:20,2128-06-27 12:32:29,2 days 21:27:09,188028,270
3,2120-08-07 23:12:42,2120-08-10 00:39:04,2 days 01:26:22,173727,271
4,2186-12-25 21:08:04,2186-12-27 12:01:13,1 days 14:53:09,164716,272
5,2141-04-19 06:12:05,2141-04-20 17:52:11,1 days 11:40:06,158689,273
6,2114-06-28 22:28:44,2114-07-07 18:01:16,8 days 19:32:32,130546,274
7,2170-10-07 11:28:53,2170-10-14 14:38:07,7 days 03:09:14,129886,275
8,2147-11-20 09:02:23,2147-11-21 17:08:52,1 days 08:06:29,135156,276
9,2132-10-21 21:11:46,2132-10-22 14:44:48,0 days 17:33:02,171601,277


In [84]:
res = bz.join(bz.join(icustays_, admissions_, 'hadm_id'), patients_, 'subject_id')
res_ = bz.transform(res, 
    age = res.admittime - res.dob
)
res_ = bz.compute(res_)

In [86]:
# rank by admittime: 
res_.groupby('subject_id')['admittime'].rank()

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       2.0
16       1.0
17       1.0
18       1.0
19       1.0
20       2.0
21       1.0
22       1.0
23       1.0
24       2.0
25       3.0
26       1.0
27       1.0
28       1.0
29       1.0
        ... 
61021    2.0
61022    1.0
61023    1.0
61024    1.0
61025    1.5
61026    1.5
61027    1.0
61028    1.0
61029    1.0
61030    1.0
61031    1.0
61032    1.0
61033    1.0
61034    1.0
61035    1.0
61036    1.0
61037    1.0
61038    2.0
61039    1.0
61040    1.0
61041    1.0
61042    1.5
61043    1.5
61044    3.0
61045    1.0
61046    1.0
61047    1.0
61048    1.0
61049    1.0
61050    1.0
Name: admittime, dtype: float64

Ok. 